In [48]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [49]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import matplotlib.pyplot as plt
import numpy as np
import os

In [50]:
dados =pd.read_csv('databasePJF.csv', header=0, sep=',', quotechar='"')

In [51]:
dados.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [52]:
#criando um dataframe com menos não fraudes para tornar maior a proporção de fraudes para não fraudes.
dadosfraud = dados[(dados.isFraud == 1)]
dadosnfraud = dados[(dados.isFraud == 0)]
dadosnfraud = dadosnfraud.sample(frac = 1)
dadosnfraud = dadosnfraud.iloc[0:25000]
dadosfinais = pd.concat([dadosfraud,dadosnfraud])
dadosfinais.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2,1,TRANSFER,181.0,C1305486145,181.0,0.0,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.0,C840083671,181.0,0.0,C38997010,21182.0,0.0,1,0
251,1,TRANSFER,2806.0,C1420196421,2806.0,0.0,C972765878,0.0,0.0,1,0
252,1,CASH_OUT,2806.0,C2101527076,2806.0,0.0,C1007251739,26202.0,0.0,1,0
680,1,TRANSFER,20128.0,C137533655,20128.0,0.0,C1848415041,0.0,0.0,1,0


In [53]:
dadosfinais = dadosfinais.sample(frac = 1)
dadosfinais = dadosfinais.reset_index()
dadosfinais.isFraud.value_counts(True)

0    0.752717
1    0.247283
Name: isFraud, dtype: float64

In [54]:
dadosfinais.columns

Index(['index', 'step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg',
       'newbalanceOrig', 'nameDest', 'oldbalanceDest', 'newbalanceDest',
       'isFraud', 'isFlaggedFraud'],
      dtype='object')

In [55]:
#utilizando o naive bayes retirado do site https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/
#utilizando a mesma base de dados utilizada para treinar o naive bayes.
dadosteste = dadosfinais[:5000]
dadostreinamento = dadosfinais[5000:]
y = dadostreinamento.isFraud
x = dadostreinamento.loc[:, [ 'step', 'amount', 'oldbalanceOrg',
       'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']]
z = dadosteste.loc[:, [ 'step', 'amount', 'oldbalanceOrg',
       'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']]
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x,y)
predicted = model.predict(z)
print(predicted)
#help(model.predict)


[0 0 0 ..., 0 0 0]


In [56]:
#comparando o real com a classficação do naive bayes
dadosteste['predicted'] = pd.Series(predicted)
dadosteste = dadosteste.loc[:, ["isFraud", "predicted"]]
print(dadosteste.isFraud.value_counts())

print(dadosteste.predicted.value_counts())
dadosteste.head()

0    3747
1    1253
Name: isFraud, dtype: int64
0    4624
1     376
Name: predicted, dtype: int64


C:\Users\vitor\Documents\Anaconda-Phython\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,isFraud,predicted
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [57]:
br = []

for a in range(len(dadosteste)):
    
    if dadosteste.isFraud[a] == 1:
        
        if dadosteste.predicted[a] == 1:
            br.append('positivos verdadeiros')
            
        if dadosteste.predicted[a] == 0:
            br.append('negativos falsos')
            
    elif dadosteste.isFraud[a] == 0:
        
        if dadosteste.predicted[a] == 0:
            br.append('negativos verdadeiros')
            
        if dadosteste.predicted[a] == 1:
            br.append('positivos falsos')


 fazer falsos positivos, verdadeiros positivos... (proj2)

In [58]:
##
porcentagem = pd.Series(br)
hue= (porcentagem.value_counts(True))*100
hue

negativos verdadeiros    73.92
negativos falsos         18.56
positivos verdadeiros     6.50
positivos falsos          1.02
dtype: float64

In [61]:
CT = pd.crosstab(dadosfinais.type,dadosfinais.isFraud)
CT

isFraud,0,1
type,,
CASH_IN,5509,0
CASH_OUT,8656,4116
DEBIT,196,0
PAYMENT,8565,0
TRANSFER,2074,4097


Em seguida Foi utilizado o método MultiNomial

In [64]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(x,y)
predicted = model.predict(z)
print(predicted)

[0 1 0 ..., 0 0 0]


In [65]:
dadosteste['predicted'] = pd.Series(predicted)
dadosteste = dadosteste.loc[:, ["isFraud", "predicted"]]
print(dadosteste.isFraud.value_counts())

print(dadosteste.predicted.value_counts())
dadosteste.head()

0    3747
1    1253
Name: isFraud, dtype: int64
1    2500
0    2500
Name: predicted, dtype: int64


,isFraud,predicted
0,0,0
1,0,1
2,0,0
3,0,0
4,0,0


In [66]:
br = []

for a in range(len(dadosteste)):
    
    if dadosteste.isFraud[a] == 1:
        
        if dadosteste.predicted[a] == 1:
            br.append('positivos verdadeiros')
            
        if dadosteste.predicted[a] == 0:
            br.append('negativos falsos')
            
    elif dadosteste.isFraud[a] == 0:
        
        if dadosteste.predicted[a] == 0:
            br.append('negativos verdadeiros')
            
        if dadosteste.predicted[a] == 1:
            br.append('positivos falsos')

In [67]:
porcentagem = pd.Series(br)
hue= (porcentagem.value_counts(True))*100
hue

negativos verdadeiros    46.56
positivos falsos         28.38
positivos verdadeiros    21.62
negativos falsos          3.44
dtype: float64